In [7]:
import numpy as np
import pandas as pd
from urllib.parse import quote_plus
import pymongo as pym

In [6]:
pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
def connect_to_mongodb():
    username = quote_plus('fanot')
    password = quote_plus('funny1cool')
    url = 'mongodb+srv://'+ username + ':' + password +'@cluster0.t4mdtbd.mongodb.net/?retryWrites=true&w=majority'
    client = pym.MongoClient(url)

    return client.get_database('Cluster0')


def recommend(name):
    recommend_restaurant = []
    db = connect_to_mongodb()
    df_percent = pd.DataFrame.from_dict(db['DF'].find())
    df_percent.set_index('name', inplace=True)
    indices = pd.Series(df_percent.index)
    idx = indices[indices == name].index[0]
    df_percent = pd.DataFrame.from_dict(db['DF'].find())
    collection = db['sim']
    result = collection.find_one()
    cosine_similarities = np.array(result['matrix'])
    # print(df_percent)
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    top31_indexes = list(score_series.iloc[0:31].index)
    for each in top31_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
        df_new = pd.DataFrame(columns=['id','address_name','name','cuisines', 'general_rating', 'avg_price'])
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['id','address_name','name','cuisines', 'general_rating', 'avg_price']][df_percent.index == each].sample()))
    df_new = df_new.drop_duplicates(subset=['cuisines', 'general_rating', 'avg_price'], keep=False)
    df_new = df_new.head(15)
    print(type(df_new))
    #print('Топ заведений с похожими отзывам: %d, %s' % (len(df_new), name))
    return df_new.to_json(orient='records', force_ascii=False)



recommend('Ватан, кафе')

#recommend('TomYumBar, кафе паназиатской кухни')

<class 'pandas.core.frame.DataFrame'>


<ipython-input-5-c094e038f651>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(pd.DataFrame(df_percent[['id','address_name','name','cuisines', 'general_rating', 'avg_price']][df_percent.index == each].sample()))
<ipython-input-5-c094e038f651>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(pd.DataFrame(df_percent[['id','address_name','name','cuisines', 'general_rating', 'avg_price']][df_percent.index == each].sample()))
<ipython-input-5-c094e038f651>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(pd.DataFrame(df_percent[['id','address_name','name','cuisines', 'general_rating', 'avg_price']][df_percent.index == each].sample()))
<ipython-input-5-c094e

'[{"id":70000001069065093,"address_name":"Смирнова, 5","name":"Ватан, кафе","cuisines":"Восточная кухня, Еврейская кухня","general_rating":4.8,"avg_price":500},{"id":70000001032775451,"address_name":"Комсомольский проспект, 70","name":"Союз, кафе","cuisines":"Русская кухня, Европейская кухня","general_rating":4.3,"avg_price":1100},{"id":70000001035530724,"address_name":"улица Котовского, 19","name":"Бульвар, кафе","cuisines":"Русская кухня, Европейская кухня","general_rating":4.5,"avg_price":1500},{"id":422740746080455,"address_name":"Иркутский тракт, 61 ст12","name":"Шустофф, ресторан","cuisines":"Русская кухня, Европейская кухня","general_rating":4.2,"avg_price":1800},{"id":70000001038511964,"address_name":"улица Кулёва, 26","name":"Santa Monica, кафе","cuisines":"Мексиканская кухня, Европейская кухня, Американская кухня","general_rating":4.3,"avg_price":1000},{"id":70000001066101507,"address_name":"Ново-Соборная площадь, 2","name":"Nova","cuisines":"Итальянская кухня, Европейская ку

In [8]:
import json
from bson import ObjectId
def getCatter():
    db = connect_to_mongodb()
    cursor = db['DF'].find()
    data = []
    for document in cursor:
        document['_id'] = str(document['_id'])  # Преобразование ObjectId в строку
        data.append(document)
    return json.loads(json.dumps(data, ensure_ascii=False))
def getCatter_dishes(id):
    db = connect_to_mongodb()
    cursor = db['dishes'].find()
    data = []
    for document in cursor:
        document['_id'] = str(document['_id'])
        data.append(document)
    return json.loads(json.dumps(data, ensure_ascii=False))


getCatter_dishes(70000001056052192)


[{'_id': '64932408298b8d1e7283ff9b',
  'id': 70000001056052192,
  'dishes': 'Шаурма "Классика" норм',
  'description': 'Ароматный лаваш, капуста, морковь, свежий огурец, МЯСО курицы, соус фирменный, маринованный лук по желанию.',
  'label': 'Шаурма',
  'images': 'https://cachizer3.2gis.com/market/ed215bda-ae18-4161-afce-a0ec2a823975.jpg',
  'price': 190,
  'vegetables': 0.47173226,
  'meat': 0.46420142,
  'seafood': 0.49533704,
  'fruits': 0.45654103,
  'mushrooms': 0.5578534,
  'grains': 0.49346703},
 {'_id': '64932408298b8d1e7283ff9c',
  'id': 70000001056052192,
  'dishes': 'Шаурма "Классика" большая',
  'description': 'Ароматный лаваш, капуста, морковь, свежий огурец, МЯСО курицы, соус фирменный, маринованный лук по желанию.',
  'label': 'Шаурма',
  'images': 'https://cachizer2.2gis.com/market/0e6cf9b5-4fb4-4483-87e8-3fbe9a7a2a70.jpg',
  'price': 230,
  'vegetables': 0.47173226,
  'meat': 0.46420142,
  'seafood': 0.49533704,
  'fruits': 0.45654103,
  'mushrooms': 0.5578534,
  'grain

In [ ]:
# df_percent = df_percent.drop(columns=['reviews_y'])
# df_percent.to_csv('df_percent.csv')